In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

#DATA_DIR = "/kaggle/input/predict-energy-behavior-of-prosumers/"
DATA_DIR = "./"

df_data = pd.read_csv(DATA_DIR + "train.csv")
df_client = pd.read_csv(DATA_DIR + "client.csv")
df_historical_weather = pd.read_csv(DATA_DIR + "historical_weather.csv")
df_forecast_weather = pd.read_csv(DATA_DIR + "forecast_weather.csv")
df_electricity_prices = pd.read_csv(DATA_DIR + "electricity_prices.csv")
df_gas_prices = pd.read_csv(DATA_DIR + "gas_prices.csv")
df_weather_station_to_county_mapping = pd.read_csv(DATA_DIR + "weather_station_to_county_mapping.csv")

Helper functions: 

In [2]:
def upsample_daily_to_hourly(df: pd.DataFrame, date_col: str) -> pd.DataFrame:
    # unsample (process) client data. Client data is reported daily, but we need it hourly.
    # So we take ever row (1 day) and duplicate (explode) it into 24 rows
    df_hourly = df.copy(deep=True) # deep copy to not make changes to dataframe from parameter
    df_hourly[date_col] = pd.to_datetime(df_hourly[date_col])

    # create a column of lists, where each list contains [00:00, 01:00, ... 23:00] for that day
    df_hourly['datetime'] = df_hourly[date_col].apply(lambda x: [x + pd.Timedelta(hours=i) for i in range(24)])
    df_hourly = df_hourly.explode('datetime')

    # drop the original daily date column as it's no longer needed
    df_hourly = df_hourly.drop(columns=[date_col])
    return df_hourly
  


def process_forecast_weather(df_forecast: pd.DataFrame, location_map: dict) -> pd.DataFrame:
    """
    Cleans, aggregates, and pivots forecast weather data.

    Logic:
    1. Map Lat/Lon to County ID.
    2. Convert 'Origin Time' (when forecast was made) to 'Target Time' (when weather happens).
    3. Group forecasts into 'batches' (Day 1 forecast vs Day 2 forecast).
    4. Average the values per county/hour/batch.
    5. Pivot so batches become columns (e.g., temperature_1, temperature_2).
    """
    # Safety Copy
    df = df_forecast.copy(deep=True)

    # 1. Map Coordinates to County
    # We use the dictionary passed in from the main function
    df['county'] = [location_map.get((x, y), -1) for x, y in zip(df['latitude'], df['longitude'])]

    # Filter out valid locations only
    df = df[df['county'] != -1]

    # 2. Calculate Target Time
    # Standardize origin time to 02:00:00 (removes minute/second noise)
    df['origin_datetime'] = pd.to_datetime(df['origin_datetime'])
    df['origin_datetime'] = pd.to_datetime(df['origin_datetime'].dt.date.astype(str) + ' 02:00:00')

    # Target Time = Origin + Hours Ahead
    df['forecast_datetime'] = df['origin_datetime'] + pd.to_timedelta(df['hours_ahead'], unit='h')

    # We don't need origin time anymore
    df.drop(columns=['origin_datetime'], inplace=True)

    # 3. Create "Batches" (cumcount) todo:rename
    # A batch represents how far out the forecast is (Day 1 vs Day 2)
    # (hours_ahead - 1) // 24 + 1 results in: 1 for 0-24h, 2 for 25-48h
    df['cumcount'] = (df['hours_ahead'] - 1) // 24 + 1

    # 4. Aggregate (Mean) by County, Time, and Batch
    # Identify feature columns (exclude IDs and Time)
    exclude_cols = ['latitude', 'longitude', 'hours_ahead', 'forecast_datetime', 'cumcount', 'county', 'data_block_id']
    feature_cols = [col for col in df.columns if col not in exclude_cols]

    agg_dict = {col: 'mean' for col in feature_cols}
    # We must keep 'cumcount' in the groupby keys, so we don't aggregate it

    df_grouped = df.groupby(['county', 'forecast_datetime', 'cumcount']).agg(agg_dict)

    # 5. Pivot (Unstack)
    # Moves 'cumcount' from a row index to a column suffix
    df_pivoted = df_grouped.unstack(level=-1)

    # Flatten MultiIndex columns: ('temperature', 1) -> 'temperature_1'
    df_pivoted.columns = [f'{col[0]}_{col[1]}' for col in df_pivoted.columns]

    df_pivoted.reset_index(inplace=True)
    df_pivoted.rename(columns={'forecast_datetime': 'datetime'}, inplace=True)

    # Handle missing values (if a forecast is missing, fill with 0 or strictly manage it)
    df_pivoted.fillna(0, inplace=True)

    return df_pivoted

def process_historical_weather(df_historical: pd.DataFrame, location_map: dict) -> pd.DataFrame:
    #compose historical weather data (averaging stations per county).
    df = df_historical.copy(deep=True)
    df['datetime'] = pd.to_datetime(df['datetime'])

    df['county'] = [location_map.get((x, y), -1) for x, y in zip(df['latitude'], df['longitude'])]
    df = df[df['county'] != -1]

    exclude_cols = ['latitude', 'longitude', 'datetime', 'county', 'data_block_id']
    agg_dict = {col: 'mean' for col in df.columns if col not in exclude_cols}
    df_grouped = df.groupby(['county', 'datetime']).agg(agg_dict)
    df_grouped.reset_index(inplace=True)

    return df_grouped

def generate_features(
    df_data: pd.DataFrame,
    df_client: pd.DataFrame,
    df_gas_prices: pd.DataFrame,
    df_electricity_prices: pd.DataFrame,
    df_historical_weather: pd.DataFrame,
    df_forecast_weather: pd.DataFrame,
    df_weather_station_to_county_mapping: pd.DataFrame,
    train_start = '2021-09-01 11:00:00'
):
  # The weather data uses Latitude/Longitude, but the energy data uses "Counties".
  # We need a dictionary to translate coordinates into county IDs so we can join them later.
  # dictionary: {(lat, lon) -> county_id}
  df_weather_station_to_county_mapping = df_weather_station_to_county_mapping[
      df_weather_station_to_county_mapping.notnull().all(axis=1)
    ].sort_values(by="county")
  result_dict = dict(zip(
      zip(
        round(df_weather_station_to_county_mapping['latitude'],1),
        round(df_weather_station_to_county_mapping['longitude'],1)),
      df_weather_station_to_county_mapping['county']))
  df_historical_weather = df_historical_weather[df_historical_weather['datetime'] >= train_start]

  # ---------------------- client data -----------------------
  df_client_hourly = upsample_daily_to_hourly(df_client, date_col='date')
  if 'data_block_id' in df_client_hourly.columns:
    df_client_hourly.drop(columns=['data_block_id'], inplace=True)

  # merge into our main dataframe
  df_data['datetime'] = pd.to_datetime(df_data['datetime'])
  df_data = df_data.merge(df_client_hourly, on=['county','product_type','is_business','datetime'], how='left')
  # Filter data to ensure we don't go past the available client data
  client_end_date = df_client_hourly['datetime'].max()
  df_data = df_data[df_data['datetime'] <= client_end_date]

  # --------------- gas prices -------------------
  df_gas_hourly = upsample_daily_to_hourly(df_gas_prices, date_col='forecast_date')
  cols_to_drop = ['origin_date', 'data_block_id']
  df_gas_hourly.drop(columns=[c for c in cols_to_drop if c in df_gas_hourly.columns], inplace=True)

  gas_end_date = df_gas_hourly['datetime'].max()
  df_data = df_data[df_data['datetime'] <= gas_end_date]

  df_data = df_data.merge(df_gas_hourly, on=['datetime'], how='left')

  #--------------- electricity (already hourly) ------------------------------
  df_electricity_prices_try = df_electricity_prices.copy(deep=True)
  if 'origin_date' in df_electricity_prices_try.columns:
    df_electricity_prices_try.drop(columns=['origin_date'], inplace=True)
  if 'data_block_id' in df_electricity_prices_try.columns:
    df_electricity_prices_try.drop(columns=['data_block_id'], inplace=True)
  df_electricity_prices_try['forecast_date'] = pd.to_datetime(df_electricity_prices_try['forecast_date'])
  df_electricity_prices_try.rename(columns={"forecast_date": "datetime"}, inplace=True)
  df_data = df_data.merge(df_electricity_prices_try, on=['datetime'], how='left')

  # --- forecast weather ---
  df_forecast_processed = process_forecast_weather(df_forecast_weather, result_dict)
  df_data = df_data.merge(df_forecast_processed, on=['county', 'datetime'], how='left')


  # --- process historical weather ---
  df_weather_processed = process_historical_weather(df_historical_weather, result_dict)
  df_data = df_data.merge(df_weather_processed, on=['county', 'datetime'], how='left')

  return df_data

In [3]:
# this is to get rid of NaN values
min_end_date = min([
    pd.to_datetime(df_client['date']).max(),
    pd.to_datetime(df_gas_prices['forecast_date']).max(),
    pd.to_datetime(df_electricity_prices['forecast_date']).max(),
    pd.to_datetime(df_historical_weather['datetime']).max(),
    pd.to_datetime(df_forecast_weather['origin_datetime']).max()
])

print(f"Filtering all data to common end date: {min_end_date}")

max_datetime = min_end_date + pd.Timedelta(hours=23)
df_data = df_data[pd.to_datetime(df_data['datetime']) <= max_datetime].copy()

print(f"df_data: {df_data['datetime'].min()} → {df_data['datetime'].max()}")
print(f"Rows: {len(df_data):,}")

Filtering all data to common end date: 2023-05-29 00:00:00
df_data: 2021-09-01 00:00:00 → 2023-05-29 23:00:00
Rows: 2,012,112


In [4]:
combined_df = generate_features(
    df_data,
    df_client,
    df_gas_prices,
    df_electricity_prices,
    df_historical_weather,
    df_forecast_weather,
    df_weather_station_to_county_mapping)

In [5]:
combined_df.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,eic_count,...,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation
0,0,0,1,0.713,0,2021-09-01,0,0,0,108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,1,96.590,1,2021-09-01,0,1,0,108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,2,0.000,0,2021-09-01,0,2,1,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,2,17.314,1,2021-09-01,0,3,1,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,3,2.904,0,2021-09-01,0,4,2,688,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
combined_df.columns

Index(['county', 'is_business', 'product_type', 'target', 'is_consumption',
       'datetime', 'data_block_id', 'row_id', 'prediction_unit_id',
       'eic_count', 'installed_capacity', 'lowest_price_per_mwh',
       'highest_price_per_mwh', 'euros_per_mwh', 'temperature_1',
       'temperature_2', 'dewpoint_1', 'dewpoint_2', 'cloudcover_high_1',
       'cloudcover_high_2', 'cloudcover_low_1', 'cloudcover_low_2',
       'cloudcover_mid_1', 'cloudcover_mid_2', 'cloudcover_total_1',
       'cloudcover_total_2', '10_metre_u_wind_component_1',
       '10_metre_u_wind_component_2', '10_metre_v_wind_component_1',
       '10_metre_v_wind_component_2', 'direct_solar_radiation_1',
       'direct_solar_radiation_2', 'surface_solar_radiation_downwards_1',
       'surface_solar_radiation_downwards_2', 'snowfall_1', 'snowfall_2',
       'total_precipitation_1', 'total_precipitation_2', 'temperature',
       'dewpoint', 'rain', 'snowfall', 'surface_pressure', 'cloudcover_total',
       'cloudcover_l

In [7]:
# Analyze NaN values
nan_summary = combined_df.isnull().sum() #checking for NaN values in each column, sum counts how many true values (NaN) in each column
nan_columns = nan_summary[nan_summary > 0].sort_values(ascending=False) #this filters only columns with NaN values and sorts them in descending order

print("NaN ANALYSIS")

if len(nan_columns) == 0: #counting the number of columns with NaN values
    print("NO NaN VALUES FOUND") 
else:
    print(f"Found NaN values in {len(nan_columns)} columns:\n") 
    for col, count in nan_columns.items(): #iterating through columns with NaN values
        pct = (count / len(combined_df)) * 100 #calculating percentage of NaN values in that column
        print(f"{col}: {count:,} ({pct:.2f}%)") #printing column name, count of NaN values and percentage
    
    print(f"\nTotal NaN: {combined_df.isnull().sum().sum():,}") #first sum counts true NaN values in each column, second sum adds them all together
    print(f"Total cells: {combined_df.size:,}")
    print(f"NaN percentage: {(combined_df.isnull().sum().sum() / combined_df.size) * 100:.2f}%")


print(f"Dataset shape: {combined_df.shape}")
print(f"Date range: {combined_df['datetime'].min()} → {combined_df['datetime'].max()}")


NaN ANALYSIS
Found NaN values in 40 columns:

winddirection_10m: 31,848 (1.58%)
shortwave_radiation: 31,848 (1.58%)
cloudcover_high: 31,848 (1.58%)
windspeed_10m: 31,848 (1.58%)
direct_solar_radiation: 31,848 (1.58%)
diffuse_radiation: 31,848 (1.58%)
cloudcover_mid: 31,848 (1.58%)
cloudcover_low: 31,848 (1.58%)
rain: 31,848 (1.58%)
snowfall: 31,848 (1.58%)
cloudcover_total: 31,848 (1.58%)
surface_pressure: 31,848 (1.58%)
temperature: 31,848 (1.58%)
dewpoint: 31,848 (1.58%)
temperature_1: 30,888 (1.54%)
temperature_2: 30,888 (1.54%)
snowfall_2: 30,888 (1.54%)
snowfall_1: 30,888 (1.54%)
cloudcover_high_2: 30,888 (1.54%)
cloudcover_high_1: 30,888 (1.54%)
dewpoint_1: 30,888 (1.54%)
dewpoint_2: 30,888 (1.54%)
cloudcover_low_1: 30,888 (1.54%)
cloudcover_low_2: 30,888 (1.54%)
10_metre_u_wind_component_2: 30,888 (1.54%)
10_metre_u_wind_component_1: 30,888 (1.54%)
surface_solar_radiation_downwards_2: 30,888 (1.54%)
surface_solar_radiation_downwards_1: 30,888 (1.54%)
direct_solar_radiation_2: 30

In [8]:
# Defining column groups so we could use them later in the loop
weather_cols = [
    'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low', 
    'cloudcover_mid', 'cloudcover_total', 'windspeed_10m', 
    'winddirection_10m', 'rain', 'snowfall', 'surface_pressure',
    'shortwave_radiation', 'direct_solar_radiation', 'diffuse_radiation'
]

# Finding all columns for predictions
forecast_cols = [col for col in combined_df.columns if col.endswith('_1') or col.endswith('_2')]


print("Handling NaN values...")
print(f"Before: {combined_df.isnull().sum().sum():,} NaN values")

# Sorting by county and time
combined_df = combined_df.sort_values(['county', 'datetime']).reset_index(drop=True)

# 1. Interpolate weather data per county - using values we already have to calculate missing values by imitating natural change/ linear change
# Works well with temperature data
for col in weather_cols + forecast_cols:
    if col in combined_df.columns:
        combined_df[col] = combined_df.groupby('county')[col].transform(
            lambda x: x.interpolate(method='linear', limit_direction='both')
        )

# 2. Fill remaining NaN with forward/backward fill - copying nearest known value until the next known value
# Works well with stable values - forward. Works well if data starts later and the first value is most likely the same - backward fill.
# Here we use both forward and backward fill to ensure no NaN values remain and we use forward first because we want to prioritize recent known values/ what was the latest known value.
combined_df[weather_cols] = combined_df.groupby('county')[weather_cols].ffill().bfill() 
combined_df[forecast_cols] = combined_df.groupby('county')[forecast_cols].ffill().bfill() 

# 3. Handling electricity prices
combined_df['euros_per_mwh'] = combined_df['euros_per_mwh'].interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')

# 4. Final safety: fill any remaining with 0 (very rare)
weather_and_forecast = [col for col in weather_cols + forecast_cols if col in combined_df.columns]
combined_df[weather_and_forecast] = combined_df[weather_and_forecast].fillna(0)
combined_df['euros_per_mwh'] = combined_df['euros_per_mwh'].fillna(combined_df['euros_per_mwh'].mean())

print(f"After: {combined_df.isnull().sum().sum():,} NaN values")
print(f"Remaining NaN in 'target': {combined_df['target'].isnull().sum()} (expected for test data)")

Handling NaN values...
Before: 1,187,978 NaN values


C:\Users\merlink0\AppData\Local\Temp\ipykernel_15380\4145414806.py:34: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df['euros_per_mwh'] = combined_df['euros_per_mwh'].interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')


After: 528 NaN values
Remaining NaN in 'target': 528 (expected for test data)


In [9]:
!pip install xgboost scikit-learn

In [11]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# FEATURE ENGINEERING

# Time features
combined_df['hour'] = pd.to_datetime(combined_df['datetime']).dt.hour
#this is improtant because energy consumption depends on the hour of the day
combined_df['day_of_week'] = pd.to_datetime(combined_df['datetime']).dt.dayofweek
#this is important because enery consumption varies between weekdays and weekends.
combined_df['month'] = pd.to_datetime(combined_df['datetime']).dt.month
#important because energy consumption can vary by month (seasonality)
combined_df['is_weekend'] = combined_df['day_of_week'].isin([5, 6]).astype(int)
# weekends have different consumption patterns

# Lag features (historical consumption)
combined_df = combined_df.sort_values(['county', 'product_type', 'is_business', 'datetime'])
combined_df['target_lag_24h'] = combined_df.groupby(['county', 'product_type', 'is_business'])['target'].shift(24)
combined_df['target_lag_168h'] = combined_df.groupby(['county', 'product_type', 'is_business'])['target'].shift(168)
#MOST important features - this is the basis of predictions for energy consumption

# Features set
improved_features = [
    'county', 'is_business', 'product_type', 'is_consumption',
    'eic_count', 'installed_capacity',
    'euros_per_mwh', 'lowest_price_per_mwh', 'highest_price_per_mwh',
    'temperature', 'cloudcover_total', 'rain',
    'temperature_1', 'temperature_2',
    'hour', 'day_of_week', 'month', 'is_weekend',
    'target_lag_24h', 'target_lag_168h',
]

# Split
train_df = combined_df[combined_df['target'].notna()].copy()
test_df = combined_df[combined_df['target'].isna()].copy()

# Removing rows with NaN in lag features (first 168 hours). This makes up abou 1.5% of the data. 
# This contain only rows where we don't have enough history to create lag features - 168 hours × county/product/business kombinatsioonid
train_df = train_df.dropna(subset=['target_lag_24h', 'target_lag_168h'])

# Sample 50% 
train_sample = train_df.sample(frac=0.5, random_state=42)
print(f"Sampled train rows: {len(train_sample):,}")

X = train_sample[improved_features].copy()
y = train_sample['target'].copy()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

#Model
params = {
    'objective': 'reg:squarederror',
    'max_depth': 6,
    'learning_rate': 0.05,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 3,
    'gamma': 0.1,
    'random_state': 42,
    'n_jobs': -1,
    'tree_method': 'hist',
    'early_stopping_rounds': 20,
}

print("\nTraining XGBoost...")
model = xgb.XGBRegressor(**params)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=20)

y_pred_train = model.predict(X_train)
y_pred_val = model.predict(X_val)

# Metrics
train_mae = mean_absolute_error(y_train, y_pred_train)
val_mae = mean_absolute_error(y_val, y_pred_val)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
val_rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))


print("MODEL PERFORMANCE")
print(f"Train MAE:  {train_mae:.4f}")
print(f"Val MAE:    {val_mae:.4f}")
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Val RMSE:   {val_rmse:.4f}")


# Feature importance
import pandas as pd
feature_importance = pd.DataFrame({
    'feature': improved_features, 
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))


# 4. PREDICTIONS ON TEST DATA

# Preparing test features
X_test = test_df[improved_features].copy()

# Checking for NaN in test features
test_nan_count = X_test.isnull().sum().sum()
if test_nan_count > 0:
    print(f" {test_nan_count} NaN values in test features")
    print("Filling NaN with 0...")
    X_test = X_test.fillna(0)

# Making predictions
test_predictions = model.predict(X_test)

# Adding predictions to test dataframe
test_df['predicted_target'] = test_predictions

print(f"\nTest predictions made: {len(test_predictions)}")
print(f"Prediction range: {test_predictions.min():.2f} to {test_predictions.max():.2f}")
print(f"Mean prediction: {test_predictions.mean():.2f}")

# Sample predictions
print("\nSample predictions:")
print(test_df[['datetime', 'county', 'product_type', 'is_business', 'predicted_target']].head(10))




Sampled train rows: 999,468

Training XGBoost...
[0]	validation_0-rmse:880.31271
[20]	validation_0-rmse:412.01673
[40]	validation_0-rmse:268.37377
[60]	validation_0-rmse:220.24941
[80]	validation_0-rmse:201.45002
[100]	validation_0-rmse:191.31822
[120]	validation_0-rmse:182.78789
[140]	validation_0-rmse:175.70998
[160]	validation_0-rmse:170.49251
[180]	validation_0-rmse:165.43575
[199]	validation_0-rmse:161.18926
MODEL PERFORMANCE
Train MAE:  67.1620
Val MAE:    68.1940
Train RMSE: 155.1686
Val RMSE:   161.1893

Top 10 Most Important Features:
               feature  importance
18      target_lag_24h    0.435558
19     target_lag_168h    0.127719
3       is_consumption    0.119138
1          is_business    0.046995
5   installed_capacity    0.041308
15         day_of_week    0.031271
12       temperature_1    0.027415
0               county    0.023491
10    cloudcover_total    0.020867
4            eic_count    0.020273

Test predictions made: 528
Prediction range: -104.38 to 6632.77


In [ ]:

# Additional improvements, trying if we can get better performance

# 1. Ading more LAG features since they are the most imprtant
combined_df = combined_df.sort_values(['county', 'product_type', 'is_business', 'datetime'])

# Rolling averages for smoother trends
combined_df['target_rolling_24h_mean'] = combined_df.groupby(['county', 'product_type', 'is_business'])['target'].transform(
    lambda x: x.shift(1).rolling(window=24, min_periods=1).mean()
)

combined_df['target_rolling_168h_mean'] = combined_df.groupby(['county', 'product_type', 'is_business'])['target'].transform(
    lambda x: x.shift(1).rolling(window=168, min_periods=1).mean()
)

# Lag 48h and 72h
combined_df['target_lag_48h'] = combined_df.groupby(['county', 'product_type', 'is_business'])['target'].shift(48)
combined_df['target_lag_72h'] = combined_df.groupby(['county', 'product_type', 'is_business'])['target'].shift(72)

# 2. Interaction features 
combined_df['temp_x_hour'] = combined_df['temperature'] * combined_df['hour']
combined_df['price_x_hour'] = combined_df['euros_per_mwh'] * combined_df['hour']
combined_df['capacity_x_consumption'] = combined_df['installed_capacity'] * combined_df['is_consumption']

# 3. Improved feature set
improved_features_v2 = [
    'county', 'is_business', 'product_type', 'is_consumption',
    'eic_count', 'installed_capacity',
    'euros_per_mwh', 'lowest_price_per_mwh', 'highest_price_per_mwh',
    'temperature', 'cloudcover_total', 'rain',
    'temperature_1', 'temperature_2',
    'hour', 'day_of_week', 'month', 'is_weekend',
    'target_lag_24h', 'target_lag_48h', 'target_lag_72h', 'target_lag_168h',
    'target_rolling_24h_mean', 'target_rolling_168h_mean',
    'temp_x_hour', 'price_x_hour', 'capacity_x_consumption',
]

# Split
train_df = combined_df[combined_df['target'].notna()].copy()
test_df = combined_df[combined_df['target'].isna()].copy()

# Removing NaN rows
train_df = train_df.dropna(subset=improved_features_v2)

# Using 70% data now (cause more is better with more features)
train_sample = train_df.sample(frac=0.7, random_state=42)
print(f"Sampled train rows: {len(train_sample):,}")

X = train_sample[improved_features_v2].copy()
y = train_sample['target'].copy()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# Improved model v2
params = {
    'objective': 'reg:squarederror',
    'max_depth': 7,  # 6 → 7
    'learning_rate': 0.03,  # 0.05 → 0.03 (slower learning rate for better convergence)
    'n_estimators': 300,  # 200 → 300
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 5,  # 3 → 5
    'gamma': 0.2,  # 0.1 → 0.2
    'reg_alpha': 0.1,  # L1 regularization
    'reg_lambda': 1.0,  # L2 regularization
    'random_state': 42,
    'n_jobs': -1,
    'tree_method': 'hist',
    'early_stopping_rounds': 30,  # 20 → 30
}

model = xgb.XGBRegressor(**params)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=20)

#Evaluation
y_pred_train = model.predict(X_train)
y_pred_val = model.predict(X_val)

train_mae = mean_absolute_error(y_train, y_pred_train)
val_mae = mean_absolute_error(y_val, y_pred_val)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
val_rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))

print("Model V2 performance")
print(f"Train MAE:  {train_mae:.4f}")
print(f"Val MAE:    {val_mae:.4f}")
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Val RMSE:   {val_rmse:.4f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': improved_features_v2,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 15 Most Important Features:")
print(feature_importance.head(15))
for idx, row in feature_importance.head(15).iterrows():
    print(f"  {row['feature']:30s} : {row['importance']:.4f} ({row['importance']*100:.2f}%)")


# Predictions with non-negative constraint
X_test = test_df[improved_features_v2].copy()
X_test = X_test.fillna(0)

test_predictions = model.predict(X_test)

# Fixing negative predictions
test_predictions = np.maximum(test_predictions, 0) 

test_df['predicted_target'] = test_predictions

print("Test predictions (non-negative)")
print(f"Predictions made: {len(test_predictions)}")
print(f"Prediction range: {test_predictions.min():.2f} to {test_predictions.max():.2f}")
print(f"Mean prediction: {test_predictions.mean():.2f}")
print(f"Negative predictions fixed: {(model.predict(X_test) < 0).sum()}")

# Sample
print("\nSample predictions:")
print(test_df[['datetime', 'county', 'product_type', 'predicted_target']].head(10))


Sampled train rows: 1,398,516
[0]	validation_0-rmse:902.91934
[20]	validation_0-rmse:524.58631
[40]	validation_0-rmse:326.07719
[60]	validation_0-rmse:230.20701
[80]	validation_0-rmse:186.34827
[100]	validation_0-rmse:165.98828
[120]	validation_0-rmse:155.88461
[140]	validation_0-rmse:149.58479
[160]	validation_0-rmse:145.30097
[180]	validation_0-rmse:141.79791
[200]	validation_0-rmse:139.60287
[220]	validation_0-rmse:137.65919
[240]	validation_0-rmse:135.98184
[260]	validation_0-rmse:134.36455
[280]	validation_0-rmse:132.67342
[299]	validation_0-rmse:131.23348
Model V2 performance
Train MAE:  42.1592
Val MAE:    43.9662
Train RMSE: 118.0630
Val RMSE:   131.2335

Top 15 Most Important Features:
                     feature  importance
19            target_lag_48h    0.494996
18            target_lag_24h    0.207594
20            target_lag_72h    0.067318
26    capacity_x_consumption    0.056321
3             is_consumption    0.044529
22   target_rolling_24h_mean    0.016621
17       